In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable


eSOH_DIR = "./data/esoh_R/"
oCV_DIR = "./data/ocv/"
fig_DIR = "./figures_fit/"
res_DIR = "./data/results_fit/"
# %matplotlib widget

In [2]:
def nmc_volume_change_mohtat(sto,c_s_max):
    t_change = -1.10/100*(1-sto)
    return t_change

def graphite_volume_change_mohtat(sto,c_s_max):
    stoichpoints = np.array([0,0.12,0.18,0.24,0.50,1])
    thicknesspoints = np.array([0,2.406/100,3.3568/100,4.3668/100,5.583/100,13.0635/100])
    x = [sto]
    t_change = pybamm.Interpolant(stoichpoints, thicknesspoints, x, name=None, interpolator='linear', extrapolate=True, entries_string=None)
    return t_change

def get_parameter_values():
    parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
    parameter_values.update(
        {
            # mechanical properties
            "Positive electrode Poisson's ratio": 0.3,
            "Positive electrode Young's modulus [Pa]": 375e9,
            "Positive electrode reference concentration for free of deformation [mol.m-3]": 0,
            "Positive electrode partial molar volume [m3.mol-1]": -7.28e-7,
            "Positive electrode volume change": nmc_volume_change_mohtat,
            # Loss of active materials (LAM) model
            "Positive electrode LAM constant exponential term": 2,
            "Positive electrode critical stress [Pa]": 375e6,
            # mechanical properties
            "Negative electrode Poisson's ratio": 0.2,
            "Negative electrode Young's modulus [Pa]": 15e9,
            "Negative electrode reference concentration for free of deformation [mol.m-3]": 0,
            "Negative electrode partial molar volume [m3.mol-1]": 3.1e-6,
            "Negative electrode volume change": graphite_volume_change_mohtat,
            # Loss of active materials (LAM) model
            "Negative electrode LAM constant exponential term": 2,
            "Negative electrode critical stress [Pa]": 60e6,
            # Other
            "Cell thermal expansion coefficient [m.K-1]": 1.48E-6,
            "Lower voltage cut-off [V]": 3.0
        },
        check_already_exists=False,
    )
    return parameter_values
parameter_values = get_parameter_values()

In [3]:
def split_long_string(title, max_words=None):
    """Get title in a nice format"""
    max_words = max_words or pybamm.settings.max_words_in_line
    words = title.split()
    # Don't split if fits on one line, don't split just for units
    if len(words) <= max_words or words[max_words].startswith("["):
        return title
    else:
        first_line = (" ").join(words[:max_words])
        second_line = (" ").join(words[max_words:])
        return first_line + "\n" + second_line

In [4]:
cell = 1


In [5]:
def load_data(cell): 
    cell_no = f'{cell:02d}'
    dfe=pd.read_csv(eSOH_DIR+"aging_param_cell_"+cell_no+".csv")
    dfe_0=pd.read_csv(eSOH_DIR+"aging_param_cell_"+cell_no+".csv")
    dfo_0=pd.read_csv(oCV_DIR+"ocv_data_cell_"+cell_no+".csv")
    dfe = dfe.drop(dfe.index[0])
    dfe = dfe.reset_index(drop=True)
    if cell_no=='13':
        dfe = dfe.drop(dfe.index[-1])
        dfe = dfe.reset_index(drop=True)
        dfe_0 = dfe_0.drop(dfe_0.index[-1])
        dfe_0 = dfe_0.reset_index(drop=True)
    dfe['N']=dfe['N']-dfe['N'][0]
    N =dfe.N.unique()
    N_0 = dfe_0.N.unique()
    print("Cycle Numbers:")
    print(*N, sep = ", ") 
    print(len(N_0))
    print(len(dfo_0))
    rev_exp = []
    irrev_exp = []

    for i in range(len(N_0)-1):
        # print(i)
        dfo = dfo_0[dfo_0['N']==N_0[i+1]]
        # print(max(dfo['E'])-min(dfo['E']))
        rev_exp.append(max(dfo['E'])-min(dfo['E']))
    dfe['rev_exp']=rev_exp
    print('Reversible Expansion')

    dfo_1 = dfo_0[dfo_0['N']==N_0[1]]
    for i in range(len(N_0)-1):
        # print(i)
        dfo = dfo_0[dfo_0['N']==N_0[i+1]]
        # print(max(dfo['E'])-min(dfo['E']))
        irrev_exp.append(min(dfo['E'])-min(dfo_1['E']))
    dfe['irrev_exp']=irrev_exp
    print('Irreversible Expansion')
    return cell_no,dfe,dfe_0,dfo_0,N,N_0

In [6]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell)

Cycle Numbers:
0, 18, 57, 93, 134, 175, 216, 257, 298, 339
11
68752
Reversible Expansion
Irreversible Expansion


In [7]:
def init_exp(cell_no,dfe):
    # dfe_0 = dfe[dfe['N']==N[0]]
    C_n_init = dfe['C_n'][0]
    C_p_init = dfe['C_p'][0]
    y_0_init = dfe['y_0'][0] 
    if cell_no=='01':
        c_rate_c = 'C/5'
        c_rate_d = 'C/5'
        dis_set = " until 3V"
    elif cell_no=='04':
        c_rate_c = '1.5C'
        c_rate_d = '1.5C'
        dis_set = " until 3V"
    elif cell_no=='07':
        c_rate_c = '2C'
        c_rate_d = '2C'
        dis_set = " until 3V"
    elif cell_no=='10':
        c_rate_c = 'C/5'
        c_rate_d = '1.5C'
        dis_set = " until 3V"
    elif cell_no=='13':
        c_rate_c = 'C/5'
        c_rate_d = 'C/5'
        dis_set = " for 150 min"
    elif cell_no=='16':
        c_rate_c = 'C/5'
        c_rate_d = '1.5C'
        dis_set = " for 20 min"

    return C_n_init,C_p_init,y_0_init,c_rate_c,c_rate_d,dis_set

In [8]:
C_n_init,C_p_init,y_0_init,c_rate_c,c_rate_d,dis_set = init_exp(cell_no,dfe)

In [28]:
pybamm.set_logging_level("WARNING")
# pybamm.set_logging_level("NOTICE")
experiment = pybamm.Experiment(
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *dfe.N.iloc[-1],
    termination="50% capacity",
#     cccv_handling="ode",
)
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        # "loss of active material": ("stress-driven","none"),
        "loss of active material": "stress-driven",
        "stress-induced diffusion": "true",
        # "lithium plating": "reversible",
    }
)
# spm.print_parameter_info()

In [10]:
param = spm.param
eps_n_data = parameter_values.evaluate(C_n_init*3600/(param.n.L * param.n.c_max * param.F* param.A_cc))
eps_p_data = parameter_values.evaluate(C_p_init*3600/(param.p.L * param.p.c_max * param.F* param.A_cc))
cs_p_init = parameter_values.evaluate(y_0_init* param.p.c_max)

In [11]:
tun_par_all=pd.read_csv(res_DIR+"fit_train_summary"+".csv")

In [12]:
tun_par=tun_par_all[tun_par_all['cell_no']==cell]
tun_par=tun_par.reset_index()

In [13]:
parameter_values = get_parameter_values()

parameter_values.update(
    {
        "SEI kinetic rate constant [m.s-1]": tun_par['k_sei'][0],
        "Positive electrode LAM constant proportional term [s-1]": tun_par['blam_p'][0],
        "Negative electrode LAM constant proportional term [s-1]": tun_par['blam_n'][0],
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16, #1.6827e-16
        # "Positive electrode LAM constant proportional term [s-1]": 4.20e-06, #4.03536e-06
        # "Negative electrode LAM constant proportional term [s-1]": 5.38e-05, #5.24755e-05
        "EC diffusivity [m2.s-1]": 2e-18,
        "Positive electrode LAM constant exponential term": 2,
        "Negative electrode LAM constant exponential term": 2,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
    },
    check_already_exists=False,
)

In [14]:
model=spm
SOC_0=1
save_at_cycles=None

In [15]:
experiment_one_cycle = pybamm.Experiment(
    experiment.operating_conditions_cycles[:1],
    termination=experiment.termination_string,
    cccv_handling=experiment.cccv_handling,
)
Vmin = 3.0
Vmax = 4.2
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = model.param
Cn = parameter_values.evaluate(param.n.cap_init)
Cp = parameter_values.evaluate(param.p.cap_init)
eps_n = parameter_values["Negative electrode active material volume fraction"]
eps_p = parameter_values["Positive electrode active material volume fraction"]
C_over_eps_n = Cn / eps_n
C_over_eps_p = Cp / eps_p
c_n_max = parameter_values.evaluate(param.n.c_max)
c_p_max = parameter_values.evaluate(param.p.c_max)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
    solver=pybamm.AlgebraicSolver(),
)

parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol[
            "x_100"
        ].data[0]
        * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol[
            "y_100"
        ].data[0]
        * c_p_max,
        
    }
)

sim_ode = pybamm.Simulation(
    model, experiment=experiment_one_cycle, parameter_values=parameter_values,
    solver=pybamm.CasadiSolver("safe")
)
sol0 = sim_ode.solve(initial_soc=SOC_0)
model = sim_ode.solution.all_models[0]
cap0 = sol0.summary_variables["Capacity [A.h]"]

2022-08-22 13:31:24.871 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/1 (27.957 ms elapsed) --------------------
2022-08-22 13:31:24.872 - [NOTICE] callbacks.on_step_start(182): Cycle 1/1, step 1/4: Discharge at C/5 until 3V
2022-08-22 13:31:25.112 - [NOTICE] callbacks.on_step_start(182): Cycle 1/1, step 2/4: Rest for 5 min
2022-08-22 13:31:25.163 - [NOTICE] callbacks.on_step_start(182): Cycle 1/1, step 3/4: Charge at C/5 until 4.2V
2022-08-22 13:31:25.379 - [NOTICE] callbacks.on_step_start(182): Cycle 1/1, step 4/4: Hold at 4.2V until C/50
2022-08-22 13:31:25.658 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.861 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2022-08-22 13:31:25.659 - [NOTICE] callbacks.on_experiment_end(222): Finish experiment simulation, took 815.209 ms


In [16]:
cap0

array([4.86098326])

In [72]:
def sol_to_y(sol, loc="end"):
    if loc == "start":
        pos = 0
    elif loc == "end":
        pos = -1
    model = sol.all_models[0]
    n_Li = sol["Total lithium in particles [mol]"].data[pos].flatten()
    Cn = sol["Negative electrode capacity [A.h]"].data[pos].flatten()
    Cp = sol["Positive electrode capacity [A.h]"].data[pos].flatten()
    # y = np.concatenate([n_Li, Cn, Cp])
    y = n_Li
    for var in model.initial_conditions:
        if var.name not in [
            "X-averaged negative particle concentration",
            "X-averaged positive particle concentration",
            "Discharge capacity [A.h]",
        ]:
            value = sol[var.name].data
            if value.ndim == 1:
                value = value[pos]
            elif value.ndim == 2:
                value = value[:, pos]
            elif value.ndim == 3:
                value = value[:, :, pos]
            y = np.concatenate([y, value.flatten()])
    return y

def y_to_sol(y, esoh_sim, model):
    print('Start y_to_sol')
    print(y[3])
    n_Li = y[0]
    Cn = C_over_eps_n * y[1]
    Cp = C_over_eps_p * y[2]

    esoh_sol = esoh_sim.solve(
        [0],
        inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li},
    )
    esoh_sim.built_model.set_initial_conditions_from(esoh_sol)
    ics = {}
    x_100 = esoh_sol["x_100"].data[0]
    y_100 = esoh_sol["y_100"].data[0]
    x_0 = esoh_sol["x_0"].data[0]
    y_0 = esoh_sol["y_0"].data[0]
    start = 1
    for var in model.initial_conditions:
        if var.name == "X-averaged negative particle concentration":
            ics[var.name] = ((x_100-x_0)*SOC_0+x_0) * np.ones((model.variables[var.name].size, 2))
        elif var.name == "X-averaged positive particle concentration":
            ics[var.name] = ((y_100-y_0)*SOC_0+y_0) * np.ones((model.variables[var.name].size, 2))
        elif var.name == "Discharge capacity [A.h]":
            ics[var.name] = np.zeros(1)
        else:
            end = start + model.variables[var.name].size
            ics[var.name] = y[start:end, np.newaxis]
            start = end
    model.set_initial_conditions_from(ics)
    print('End y_to_sol')
    print(pybamm.Solution(

        [np.array([0])],
        model.concatenated_initial_conditions.evaluate()[:, np.newaxis],
        model,
        {},
    ))
    return pybamm.Solution(
        [np.array([0])],
        model.concatenated_initial_conditions.evaluate()[:, np.newaxis],
        model,
        {},
    )

def dydt(t, y):
    print('Start dydt')
    if y[0] < 0 or y[1] < 0 or y[2] < 0:
        return 0 * y

    print(t)
    # Set up based on current value of y
    y_to_sol(
        y,
        esoh_sim,
        sim_ode.op_conds_to_built_models[
            experiment_one_cycle.operating_conditions[0]["electric"]
        ],
    )
    # Simulate one cycle
    print("before sol")
    sol = sim_ode.solve()
    print("After sol")
    dy = sol_to_y(sol) - y
    print('End dydt')
    return dy


In [23]:
if experiment.termination == {}:
    event = None
    print('1')
else:
    print('2')
    def capacity_cutoff(t, y):
        sol = y_to_sol(y, esoh_sim, model)
        cap = pybamm.make_cycle_solution([sol], esoh_sim, True)[1]["Capacity [A.h]"]
        return cap / cap0 - experiment_one_cycle.termination["capacity"][0] / 100

    capacity_cutoff.terminal = True

2


In [30]:
num_cycles = len(experiment.operating_conditions_cycles)
if save_at_cycles is None:
    t_eval = np.arange(1, num_cycles + 1)
elif save_at_cycles == -1:
    t_eval = None
else:
    t_eval = np.arange(1, num_cycles + 1, save_at_cycles)
y0 = sol_to_y(sol0, loc="start")
timer = pybamm.Timer()

In [31]:
y0 = sol_to_y(sol0, loc="start")

array([0.18896994, 0.59957373, 0.43573014, 1.        ])

In [73]:
sol = solve_ivp(
    dydt,
    [1, num_cycles],
    y0,
    t_eval=t_eval,
    events=capacity_cutoff,
    first_step=10,
    method="RK23",
    atol=1e-2,
    rtol=1e-2,
)
time = timer.time()

Start dydt
1.0
Start y_to_sol
1.0
End y_to_sol
before sol
After sol
End dydt
Start y_to_sol
1.0
End y_to_sol


ValueError: diff requires input that is at least one dimensional

In [56]:
dydt(1,y0)

1
1.0


array([-1.52436701e-04, -3.99559452e-04, -6.50811218e-05,  8.97818577e-02])

In [63]:
sol = solve_ivp(
    dydt,
    [1, num_cycles],
    y0,
    t_eval=t_eval,
    events=capacity_cutoff,
    first_step=10,
    method="RK23",
    atol=1e-2,
    rtol=1e-2,
)
time = timer.time()

1.0
1.0
[-1.52436701e-04 -3.99559452e-04 -6.50811218e-05  8.97818577e-02]
1.0


ValueError: diff requires input that is at least one dimensional

In [ ]:



all_sumvars = []
for idx in range(sol.y.shape[1]):
    fullsol = y_to_sol(sol.y[:, idx], esoh_sim, model)
    sumvars = pybamm.make_cycle_solution([fullsol], esoh_sim, True)[1]
    all_sumvars.append(sumvars)

all_sumvars_dict = {
    key: np.array([sumvars[key] for sumvars in all_sumvars])
    for key in all_sumvars[0].keys()
}
all_sumvars_dict["Cycle number"] = sol.t

all_sumvars_dict["cycles evaluated"] = sol.nfev
all_sumvars_dict["solution time"] = time